In [1]:
from google.colab import files
files.upload() # kaggle.jsonをアップロード
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
import sys
import numpy as np
import pandas as pd 
from sklearn.svm import SVC
# 標準化、正規化用のクラスをインポート
from sklearn.preprocessing import RobustScaler, \
                   MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# データの準備
def prepare():
    !kaggle datasets download -d \
    iabhishekofficial/mobile-price-classification
    !unzip mobile-price-classification.zip 
    data = pd.read_csv('train.csv')
    # 分類に使用する特徴量
    features = []
    for f in data.columns.values:
        if f != 'price_range':
            features.append(f)
    X = data.loc[:, features].values
    y = data.loc[:, ['price_range'] ].values.ravel()
    return X, y, features
# 標準化・正規化(3種類の方法)
def preprocess(X_train, X_test, scaler_type):
    if scaler_type == 'robust':
        scaler = RobustScaler()
    elif scaler_type == 'standard':
        scaler = StandardScaler()
    elif scaler_type == 'minmax':
        scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_rs = scaler.transform(X_train)
    X_test_rs = scaler.transform(X_test)
    return X_train_rs, X_test_rs

# SVMでのモデル学習および予測と評価
def svm_test(X_train, y_train, X_test, y_test):
    clf = SVC()
    clf.fit(X_train, y_train) 
    y_pred = clf.predict(X_test)
    print('\nClassification Accuracy: %.3lf' % \
              accuracy_score(y_test, y_pred))
def main():
    X_train, y_train, features = prepare()
    X_train, X_test, y_train, y_test = train_test_split( \
         X_train, y_train, train_size=0.7, random_state=1)
    # 通常の標準化を行い、SVMで分類し、Accuracy算出
    X_train_sc, X_test_sc = \
          preprocess(X_train, X_test, 'standard')
    print('<<StandardScaler>>')
    print(X_train_sc) 
    svm_test(X_train_sc, y_train, X_test_sc, y_test)
    # MinMaxScalerで正規化を行った後、
    # SVMで学習・分類し、Accuracy算出
    X_train_ms, X_test_ms = \
          preprocess(X_train, X_test,'minmax')
    print('<<MinMaxScaler>>')
    print(X_train_ms)
    svm_test(X_train_ms, y_train, X_test_ms, y_test)
    # 外れ値に頑健な標準化(RobustScaler)を行い、
    # SVMでモデルを学習し、予測結果からAccuracy算出
    X_train_rs, X_test_rs = \
          preprocess(X_train, X_test, 'robust')
    print('<<RobustScaler>>')
    print(X_train_rs) 
    svm_test(X_train_rs, y_train, X_test_rs, y_test)

if __name__ == '__main__':
    main()
    

  0% 0.00/70.6k [00:00<?, ?B/s]
100% 70.6k/70.6k [00:00<00:00, 25.8MB/s]
Archive:  mobile-price-classification.zip
  inflating: test.csv                
  inflating: train.csv               
<<StandardScaler>>
[[ 0.12296692 -1.00142959  1.58033471 ... -1.73867435  0.98722446
   1.00286124]
 [ 0.6169352  -1.00142959  1.70309277 ... -1.73867435  0.98722446
   1.00286124]
 [-0.21936458  0.99857245  1.45757665 ...  0.57515083  0.98722446
  -0.99714693]
 ...
 [-1.2118962   0.99857245  0.10723794 ...  0.57515083  0.98722446
   1.00286124]
 [ 0.63761294  0.99857245  0.35275407 ...  0.57515083  0.98722446
  -0.99714693]
 [ 0.63531542  0.99857245 -0.99758464 ...  0.57515083 -1.01294087
   1.00286124]]

Classification Accuracy: 0.857
<<MinMaxScaler>>
[[0.53244147 0.         0.92       ... 0.         1.         1.        ]
 [0.67625418 0.         0.96       ... 0.         1.         1.        ]
 [0.43277592 1.         0.88       ... 1.         1.         0.        ]
 ...
 [0.14381271 1.         0